In [1]:
!pip install termcolor
!pip install pandas_ta

In [2]:
from kiteconnect import KiteConnect
import urllib.parse
import tqdm
from requests import auth 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os, time
from os.path import expanduser
from selenium.webdriver.chrome.options import Options
import pyotp
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import pdb

import sys
import hmac, base64, struct, hashlib, time
import requests


zerodha_cred = {
    'User':{
        'user_id' : 'RK2267',
        'password': "Shan@#2021",
        'api_key' : "pv2830q1vbrhu1eu",   
        'common_ans' : "",
        'api_secret' :'gz426yatawim9xruj2p6e51c40k44r7n',  
        'auth_key': 'TOOAIR75STSUWM67LS3NVUX7B2LGKJ7R'
    },

}
my_api_key=zerodha_cred['User']['api_key']

'''def telegram_bot_sendtext(my_message='SYSTEM RUNNING \n ------------------------- \nNo input received from program'):
    bot_token = '2020880061:AAHDR4kpcb4Y7lsbq59uUpgcqq7hiHYK_AE'  # ParkingBot ID
    bot_chatID = '-746818656'  # TelegramGroup ID 746818656

    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + my_message
    response = requests.get(send_text)
    return response.json()

telegram_bot_sendtext("Starting Zerodha Auth Key fetching")'''


def get_hotp_token(secret, intervals_no):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", intervals_no)
    h = hmac.new(key, msg, hashlib.sha1).digest()
    o = h[19] & 15
    h = (struct.unpack(">I", h[o:o+4])[0] & 0x7fffffff) % 1000000
    return h

def get_totp_token(secret):
    totp = str(get_hotp_token(secret, intervals_no=int(time.time())//30))
    if len(totp) < 6:
        totp = '0' + totp
    return totp


def getkite(_client):

    user_id = zerodha_cred[_client]['user_id']
    password = zerodha_cred[_client]['password']
    api_key = zerodha_cred[_client]['api_key']
    kite = KiteConnect(api_key=api_key)
    # common_ans = zerodha_cred[_client]['common_ans']
    auth_key = zerodha_cred[_client]['auth_key']
    url  = 'https://kite.trade/connect/login?v=3&api_key='+api_key 
    # telegram_bot_sendtext("Line 92")

    try:
        options = Options()
        options.add_argument('--no-sandbox')
        options.add_argument('--headless')
        options.add_argument('--disble-dev-shm-usage')
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        time.sleep(2)
        input_fields = driver.find_elements_by_tag_name('input')
        input_fields[0].send_keys(user_id)
        input_fields[1].send_keys(password)
        input_fields[1].send_keys(Keys.ENTER)
        time.sleep(1)
        two_fa = driver.find_elements_by_tag_name('input')
        common_ans = get_totp_token(auth_key)
        two_fa[0].send_keys(str(common_ans))
        two_fa[0].send_keys(Keys.ENTER)
        time.sleep(1)
        while("request_token=" not in driver.current_url):
            print (driver.current_url)
            driver.save_screenshot('login.png')
            time.sleep(3)
        redirect_url = driver.current_url
        #print('redirectURL-',redirect_url)
        parsed = urllib.parse.urlparse(redirect_url)
        query_dict = dict(urllib.parse.parse_qsl(parsed.query))
        request_token = query_dict['request_token']
        driver.close
        return request_token
    except Exception as e:
        print('closing')
        driver.close()
def gen_acc_token():
    global acc_key
    users = ['User'] 
    auth_keys={}
    cur_date=datetime.now().strftime("%y_%m_%d")

    for _client in users:
        api_key = zerodha_cred[_client]['api_key']
        api_secret = zerodha_cred[_client]['api_secret']

        req_token=getkite(_client)  
        # req_token = '8DoJtcPaDkOEj0bxCOm4KC7u4Y3QUd8H'
        url  = 'https://kite.trade/connect/login?v=3&api_key='+api_key 
        print ('url-',url)
        # pdb.set_trace()
        # telegram_bot_sendtext("Line 186")
        # pdb.set_trace()
        kite = KiteConnect(api_key=api_key)
        data = kite.generate_session(req_token, api_secret=api_secret)
        access_token=data["access_token"]
        print(data['user_name'],'has successfully signed in.')
        acc_key=access_token
        print('Access_token->',access_token)
        auth_keys[_client] = access_token

        auth_keys['date'] = cur_date
        telegram_bot_sendtext("*Auth keys extracted!*\n\nUser = \t\t" + _client   +  "\nAuth Key = \t" + auth_keys[_client] +  "\nDate = \t" + auth_keys['date'])

if __name__ == '__main__':
    try:
        gen_acc_token()
        acc_key=""
    except Exception as e:
        print('Happy Trading !')
        #print(my_api_key,acc_key)
        #telegram_bot_sendtext(f" Error in Token Generation - {str(e)}")

url- https://kite.trade/connect/login?v=3&api_key=pv2830q1vbrhu1eu
Kakal Krishna Rao has successfully signed in.
Access_token-> 0zbanc5UBbmP5dsgDuTnBPoGCi458BU6
Happy Trading !


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))

In [4]:
from datetime import *

def history_data_cpr(ticker_val):
    data1=['' for i in range(1)]
    start_date=(datetime.now()-timedelta(days=1029))
    new_date=(datetime.now())
    
    startd=pd.to_datetime(str(start_date)[:10])
    endd=pd.to_datetime(str(new_date))
    #print(startd,endd)
    
    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/day?from="+str(startd)+"&to="+str(endd)
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    
    data1[i]=pd.DataFrame(data1[i])
    
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print('History_data {} collected .'.format(ticker_val))
    return data1

In [5]:
# Read xls
import pandas as pd

trades=pd.read_csv('Trades.csv')
trades

,TICKER,TOKEN,TARGET1,TARGET2,STOPLOSS,TYPE
0,ACC,5633,2650,2700,2530,LONG
1,BHARTIARTL,2714625,801,780,861,SHORT
2,CIPLA,177665,1080,1060,1160,SHORT
3,DLF,3771393,420,430,390,LONG
4,TATAPOWER,877057,233,238,220,LONG
5,BAJFINANCE,81153,6610,6500,6800,SHORT
6,ASHOKLEY,54273,142,138,149,SHORT
7,BHARATFORG,108033,820,800,860,SHORT
8,BAJAJFINSV,4268801,1550,1500,1660,SHORT
9,MUTHOOTFIN,6054401,1140,1180,1080,LONG


In [6]:
trades_dict=trades.to_dict(orient='index')
trades_dict


{0: {'TICKER': 'ACC',
  'TOKEN': 5633,
  'TARGET1': 2650,
  'TARGET2': 2700,
  'STOPLOSS': 2530,
  'TYPE': 'LONG'},
 1: {'TICKER': 'BHARTIARTL',
  'TOKEN': 2714625,
  'TARGET1': 801,
  'TARGET2': 780,
  'STOPLOSS': 861,
  'TYPE': 'SHORT'},
 2: {'TICKER': 'CIPLA',
  'TOKEN': 177665,
  'TARGET1': 1080,
  'TARGET2': 1060,
  'STOPLOSS': 1160,
  'TYPE': 'SHORT'},
 3: {'TICKER': 'DLF',
  'TOKEN': 3771393,
  'TARGET1': 420,
  'TARGET2': 430,
  'STOPLOSS': 390,
  'TYPE': 'LONG'},
 4: {'TICKER': 'TATAPOWER',
  'TOKEN': 877057,
  'TARGET1': 233,
  'TARGET2': 238,
  'STOPLOSS': 220,
  'TYPE': 'LONG'},
 5: {'TICKER': 'BAJFINANCE',
  'TOKEN': 81153,
  'TARGET1': 6610,
  'TARGET2': 6500,
  'STOPLOSS': 6800,
  'TYPE': 'SHORT'},
 6: {'TICKER': 'ASHOKLEY',
  'TOKEN': 54273,
  'TARGET1': 142,
  'TARGET2': 138,
  'STOPLOSS': 149,
  'TYPE': 'SHORT'},
 7: {'TICKER': 'BHARATFORG',
  'TOKEN': 108033,
  'TARGET1': 820,
  'TARGET2': 800,
  'STOPLOSS': 860,
  'TYPE': 'SHORT'},
 8: {'TICKER': 'BAJAJFINSV',
  'TO

In [7]:
trades_tokens={}
for i in range(len(trades)):
    trades_tokens[int(trades_dict[i]['TOKEN'])]=i
trades_tokens

{5633: 0,
 2714625: 1,
 177665: 2,
 3771393: 3,
 877057: 4,
 81153: 5,
 54273: 6,
 108033: 7,
 4268801: 8,
 6054401: 9,
 1207553: 10}

In [8]:
from datetime import *
def history_data(ticker_val):
    data1=['' for i in range(1)]
    t2=(datetime.now() - timedelta(days=0))
    t1=(datetime.now() - timedelta(days=1))
    curr_hr=str(t2).split(':')[0].split(' ')[-1]
    curr_min=str(t2).split(':')[1]
    curr_secs=str(t2).split(':')[2]
    final=curr_hr+':'+curr_min+':'+curr_secs

    t1=str(t1)
    t2=str(t2)
    t1=t1.split(" ")
    t3=t1[1].split(':')
    t3[0]='9'
    t3[1]='00'
    t3=t3[0]+':'+t3[1]+':'+t3[2]
    t1=t1[0]+"+"+t3
    t2=t2.split(" ")
    t3=t2[1].split(':')
    t3[0]='9'
    t3[1]='00'
    t3=t3[0]+':'+t3[1]+':'+t3[2]
    t2=t2[0]+"+"+final
    t1,t2

    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/minute?from="+t1+"&to="+t2
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    data1[i]=pd.DataFrame(data1[i])
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print('History_data {} collected .'.format(ticker_val))
    return data1

In [9]:
import threading
import logging
from kiteconnect import KiteTicker
from termcolor import colored
import time
import pywhatkit
import tqdm
import pandas as pd
from datetime import datetime
from datetime import timedelta
from multiprocessing.pool import ThreadPool

rex=set()
mx=set()
st=set()
kws = KiteTicker(my_api_key, acc_key)
#dicct={4267265: 'BAJAJ-AUTO'}
dicct={5633: 'ACC', 6401: 'ADANIENT', 3861249: 'ADANIPORTS', 325121: 'AMBUJACEM', 40193: 'APOLLOHOSP', 54273: 'ASHOKLEY', 60417: 'ASIANPAINT', 1510401: 'AXISBANK', 4267265: 'BAJAJ-AUTO', 4268801: 'BAJAJFINSV', 81153: 'BAJFINANCE', 579329: 'BANDHANBNK', 1195009: 'BANKBARODA', 98049: 'BEL', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 140033: 'BRITANNIA', 2763265: 'CANBK', 177665: 'CIPLA', 5215745: 'COALINDIA', 1215745: 'CONCOR', 486657: 'CUMMINSIND', 5105409: 'DEEPAKNTR', 2800641: 'DIVISLAB', 3771393: 'DLF', 225537: 'DRREDDY', 232961: 'EICHERMOT', 261889: 'FEDERALBNK', 315393: 'GRASIM', 589569: 'HAL', 1850625: 'HCLTECH', 340481: 'HDFC', 341249: 'HDFCBANK', 119553: 'HDFCLIFE', 345089: 'HEROMOTOCO', 348929: 'HINDALCO', 356865: 'HINDUNILVR', 1270529: 'ICICIBANK', 2863105: 'IDFCFIRSTB', 387073: 'INDHOTEL', 1346049: 'INDUSINDBK', 408065: 'INFY', 3484417: 'IRCTC', 424961: 'ITC', 1723649: 'JINDALSTEL', 3001089: 'JSWSTEEL', 4632577: 'JUBLFOOD', 492033: 'KOTAKBANK', 511233: 'LICHSGFIN', 2939649: 'LT', 4561409: 'LTI', 519937: 'M&M', 2815745: 'MARUTI', 582913: 'MRF', 6054401: 'MUTHOOTFIN', 2977281: 'NTPC', 633601: 'ONGC', 2730497: 'PNB', 3834113: 'POWERGRID', 4708097: 'RBLBANK', 738561: 'RELIANCE', 758529: 'SAIL', 4600577: 'SBICARD', 779521: 'SBIN', 794369: 'SHREECEM', 837889: 'SRF', 857857: 'SUNPHARMA', 871681: 'TATACHEM', 884737: 'TATAMOTORS', 877057: 'TATAPOWER', 895745: 'TATASTEEL', 2953217: 'TCS', 3465729: 'TECHM', 897537: 'TITAN', 2170625: 'TVSMOTOR', 2952193: 'ULTRACEMCO', 2889473: 'UPL', 784129: 'VEDL', 951809: 'VOLTAS', 969473: 'WIPRO', 975873: 'ZEEL'}
tokens=list(dicct.keys())



url='https://api.telegram.org/bot5947670273:AAHjW84ezhBgwDl4R9fYSfAD3WQTS7v-Bc4/sendMessage?chat_id=-688546096&text='
'''tokens=[60750087,59897095,60128519]
dicct={60750087:'SILVERMIC',59897095:'SILVERM',60128519:'SILVER'}'''
flag=False
x=-1
x2=-1
s=""
enter=False
count=0

    

            
def calculate(tkname,s,ltp,ohlc):
    global rex,x,dicct,tokens,mx,flag,st,cpr_1day_dict,enter
    #print(tx)
    if x^(s.minute)==0:
        if (s.second==59 or s.second==58) and tkname not in st:
            st.add(tkname)
            if tkname in trades_tokens.keys():
                
                if trades_dict[trades_tokens[tkname]]['TYPE']=='LONG':
                
                    if ltp>=trades_dict[trades_tokens[tkname]]['TARGET1']:
                        k=trades_dict[trades_tokens[tkname]]['TARGET1']
                        #pywhatkit.sendwhatmsg(target_no,f'{dicct[tkname]} TARGET {k} ACHEIVED',s.hour,s.minute)
                        requests.get(url+f'{dicct[tkname]} TARGET1 {k} ACHEIVED !',stream=True)
                        print(f'{dicct[tkname]} TARGET1 {k} ACHEIVED')
                        trades_dict[trades_tokens[tkname]]['TARGET1']=float('inf')
                        
                        
                    if ltp>=trades_dict[trades_tokens[tkname]]['TARGET2']:
                        k=trades_dict[trades_tokens[tkname]]['TARGET2']
                        #pywhatkit.sendwhatmsg(target_no,f'{dicct[tkname]} TARGET {k} ACHEIVED',s.hour,s.minute)
                        requests.get(url+f'{dicct[tkname]} TARGET2 {k} ACHEIVED !',stream=True)
                        trades_dict[trades_tokens[tkname]]['TARGET2']=float('-inf')
                        print(f'{dicct[tkname]} TARGET2 {k} ACHEIVED')
                    
                    if ltp <= trades_dict[trades_tokens[tkname]]['STOPLOSS']:
                        m=trades_dict[trades_tokens[tkname]]['STOPLOSS']
                        #pywhatkit.sendwhatmsg(target_no,f'{dicct[tkname]} STOPLOSS {m} ACHEIVED',s.hour,s.minute)
                        requests.get(url+f'{dicct[tkname]} STOPLOSS {m} ACHEIVED',stream=True)
                        trades_dict[trades_tokens[tkname]]['STOPLOSS']=float('inf')
                        print(f'{dicct[tkname]} STOPLOSS {m} ACHEIVED')
                    
                elif trades_dict[trades_tokens[tkname]]['TYPE']=='SHORT':
                    
                    if ltp<=trades_dict[trades_tokens[tkname]]['TARGET1']:
                        k=trades_dict[trades_tokens[tkname]]['TARGET1']
                        #pywhatkit.sendwhatmsg(target_no,f'{dicct[tkname]} TARGET {k} ACHEIVED',s.hour,s.minute)
                        requests.get(url+f'{dicct[tkname]} TARGET1 {k} ACHEIVED !',stream=True)
                        trades_dict[trades_tokens[tkname]]['TARGET1']=float('inf')
                        print(f'{dicct[tkname]} TARGET1 {k} ACHEIVED')
                        
                    if ltp<=trades_dict[trades_tokens[tkname]]['TARGET2']:
                        k=trades_dict[trades_tokens[tkname]]['TARGET2']
                        #pywhatkit.sendwhatmsg(target_no,f'{dicct[tkname]} TARGET {k} ACHEIVED',s.hour,s.minute)
                        requests.get(url+f'{dicct[tkname]} TARGET2 {k} ACHEIVED !',stream=True)
                        trades_dict[trades_tokens[tkname]]['TARGET2']=float('inf')
                        print(f'{dicct[tkname]} TARGET2 {k} ACHEIVED')
                    
                    if ltp >= trades_dict[trades_tokens[tkname]]['STOPLOSS']:
                        m=trades_dict[trades_tokens[tkname]]['STOPLOSS']
                        #pywhatkit.sendwhatmsg(target_no,f'{dicct[tkname]} STOPLOSS {m} ACHEIVED',s.hour,s.minute)
                        requests.get(url+f'{dicct[tkname]} STOPLOSS {m} ACHEIVED',stream=True)
                        trades_dict[trades_tokens[tkname]]['STOPLOSS']=float('-inf')
                        print(f'{dicct[tkname]} STOPLOSS {m} ACHEIVED')
                    
            res={}
            res={'Time':s,'Ticker': dicct[tkname],'Close': ltp}
            print(res)
    else:
        x=s.minute
        #globals()['prevclose'+'_{}'.format(tkname)]=ltp
        st=set()
        print("-------------------------START OF {} mins-----------------------".format(x))
    
    

            
def on_ticks(ws, ticks):
    logging.debug("Ticks: {}".format(ticks))
    tim=-1
    thread_list=[]
    for turn in range(len(ticks)):
        ltp=ticks[turn]['last_price']
        ohlc=ticks[turn]['ohlc']
        T2=datetime.now()
        hr2=int(T2.hour)
        min2=int(T2.minute)
        sec2=int(T2.second)
        tk=ticks[turn]['instrument_token']
        #T4=str(T2).split(' ')[0]
        TR1=str(T2).split(' ')[1]
        s=pd.to_datetime(TR1)
        try:
            t1=threading.Thread(target=calculate,args=[tk,s,ltp,ohlc])
            #t2=threading.Thread(target=calculate,args=[tk,s,ltp,ohlc])
            t1.start()
            t1.join()
        except:
            print('Error1')
            continue
        
        
def on_connect(ws, response):
    print("Trading starting soon....")
    check_flag=False
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL, tokens)
    print("Daily Trading Started. Good Luck....")
    requests.get(url+'TARGET-STOPLOSS notebook executed !',stream=True)


          
            
def on_close(ws, code, reason):

    # kws.enable_reconnect(reconnect_interval=5, reconnect_tries=50)
    # print("reconnecting...")
    requests.get(url+'Closing Connection',stream=True)
    print("closing connection")
    logging.info("Connection closed: {code} - {reason}".format(code=code, reason=reason))
    # ws.stop()

def on_error(ws, code, reason):
    logging.info("123456Connection error: {code} - {reason}".format(code=code, reason=reason))


def on_reconnect(ws, attempts_count):
    logging.info("123456Reconnecting: {}".format(attempts_count))


def on_noreconnect(ws):
    logging.info("123456Reconnect failed.")


# Assign the callbacks.
kws.on_ticks = on_ticks

kws.on_close = on_close
kws.on_error = on_error
kws.on_connect = on_connect
kws.on_reconnect = on_reconnect
kws.on_noreconnect = on_noreconnect
kws.connect(threaded=True, disable_ssl_verification=False)


Trading starting soon....
Daily Trading Started. Good Luck....
-------------------------START OF 58 mins-----------------------
{'Time': Timestamp('2022-12-06 12:58:58.150905'), 'Ticker': 'ITC', 'Close': 337.5}
{'Time': Timestamp('2022-12-06 12:58:58.152185'), 'Ticker': 'MUTHOOTFIN', 'Close': 1119.15}
{'Time': Timestamp('2022-12-06 12:58:58.153067'), 'Ticker': 'SHREECEM', 'Close': 24000.0}
{'Time': Timestamp('2022-12-06 12:58:58.153513'), 'Ticker': 'BEL', 'Close': 106.1}
{'Time': Timestamp('2022-12-06 12:58:58.153863'), 'Ticker': 'DLF', 'Close': 413.1}
{'Time': Timestamp('2022-12-06 12:58:58.154247'), 'Ticker': 'ULTRACEMCO', 'Close': 7190.0}
{'Time': Timestamp('2022-12-06 12:58:58.154581'), 'Ticker': 'ACC', 'Close': 2626.4}
{'Time': Timestamp('2022-12-06 12:58:58.155025'), 'Ticker': 'ADANIPORTS', 'Close': 894.65}
{'Time': Timestamp('2022-12-06 12:58:58.155511'), 'Ticker': 'DRREDDY', 'Close': 4399.25}
{'Time': Timestamp('2022-12-06 12:58:58.155993'), 'Ticker': 'ICICIBANK', 'Close': 923.

{'Time': Timestamp('2022-12-06 12:59:58.548852'), 'Ticker': 'APOLLOHOSP', 'Close': 4746.6}
{'Time': Timestamp('2022-12-06 12:59:58.550891'), 'Ticker': 'IDFCFIRSTB', 'Close': 60.65}
{'Time': Timestamp('2022-12-06 12:59:58.551589'), 'Ticker': 'MRF', 'Close': 94494.8}
{'Time': Timestamp('2022-12-06 12:59:58.552569'), 'Ticker': 'NTPC', 'Close': 174.4}
{'Time': Timestamp('2022-12-06 12:59:58.553226'), 'Ticker': 'HINDUNILVR', 'Close': 2644.95}
{'Time': Timestamp('2022-12-06 12:59:58.554546'), 'Ticker': 'ADANIENT', 'Close': 4013.0}
{'Time': Timestamp('2022-12-06 12:59:58.555224'), 'Ticker': 'HEROMOTOCO', 'Close': 2785.3}
{'Time': Timestamp('2022-12-06 12:59:58.556100'), 'Ticker': 'INFY', 'Close': 1614.1}
{'Time': Timestamp('2022-12-06 12:59:58.556797'), 'Ticker': 'COALINDIA', 'Close': 230.2}
{'Time': Timestamp('2022-12-06 12:59:58.557167'), 'Ticker': 'HCLTECH', 'Close': 1112.8}
{'Time': Timestamp('2022-12-06 12:59:58.558554'), 'Ticker': 'VEDL', 'Close': 313.4}
{'Time': Timestamp('2022-12-06 1

{'Time': Timestamp('2022-12-06 13:00:59.766812'), 'Ticker': 'IDFCFIRSTB', 'Close': 60.55}
{'Time': Timestamp('2022-12-06 13:00:59.776649'), 'Ticker': 'DIVISLAB', 'Close': 3352.7}
{'Time': Timestamp('2022-12-06 13:00:59.780286'), 'Ticker': 'LICHSGFIN', 'Close': 404.0}
{'Time': Timestamp('2022-12-06 13:00:59.781528'), 'Ticker': 'ONGC', 'Close': 141.65}
-------------------------START OF 1 mins-----------------------
{'Time': Timestamp('2022-12-06 13:01:58.014602'), 'Ticker': 'ADANIENT', 'Close': 4013.8}
{'Time': Timestamp('2022-12-06 13:01:58.016574'), 'Ticker': 'CANBK', 'Close': 321.35}
{'Time': Timestamp('2022-12-06 13:01:58.017602'), 'Ticker': 'HEROMOTOCO', 'Close': 2789.8}
{'Time': Timestamp('2022-12-06 13:01:58.018227'), 'Ticker': 'ZEEL', 'Close': 265.15}
{'Time': Timestamp('2022-12-06 13:01:58.018598'), 'Ticker': 'BHEL', 'Close': 88.25}
{'Time': Timestamp('2022-12-06 13:01:58.018951'), 'Ticker': 'COALINDIA', 'Close': 230.25}
{'Time': Timestamp('2022-12-06 13:01:58.019289'), 'Ticker'

{'Time': Timestamp('2022-12-06 13:02:59.176246'), 'Ticker': 'M&M', 'Close': 1257.45}
{'Time': Timestamp('2022-12-06 13:02:59.179114'), 'Ticker': 'CANBK', 'Close': 321.65}
{'Time': Timestamp('2022-12-06 13:02:59.181444'), 'Ticker': 'VEDL', 'Close': 313.9}
{'Time': Timestamp('2022-12-06 13:02:59.181805'), 'Ticker': 'CONCOR', 'Close': 760.55}
{'Time': Timestamp('2022-12-06 13:02:59.182160'), 'Ticker': 'DIVISLAB', 'Close': 3356.55}
{'Time': Timestamp('2022-12-06 13:02:59.182533'), 'Ticker': 'HDFC', 'Close': 2666.8}
{'Time': Timestamp('2022-12-06 13:02:59.183518'), 'Ticker': 'BANDHANBNK', 'Close': 244.45}
{'Time': Timestamp('2022-12-06 13:02:59.184799'), 'Ticker': 'UPL', 'Close': 788.7}
{'Time': Timestamp('2022-12-06 13:02:59.185480'), 'Ticker': 'ASIANPAINT', 'Close': 3153.0}
{'Time': Timestamp('2022-12-06 13:02:59.186172'), 'Ticker': 'GRASIM', 'Close': 1809.15}
{'Time': Timestamp('2022-12-06 13:02:59.186846'), 'Ticker': 'RBLBANK', 'Close': 154.25}
{'Time': Timestamp('2022-12-06 13:02:59.18

{'Time': Timestamp('2022-12-06 13:03:59.745599'), 'Ticker': 'VEDL', 'Close': 314.0}
{'Time': Timestamp('2022-12-06 13:03:59.749908'), 'Ticker': 'JUBLFOOD', 'Close': 551.8}
-------------------------START OF 4 mins-----------------------
{'Time': Timestamp('2022-12-06 13:04:58.139379'), 'Ticker': 'M&M', 'Close': 1257.05}
{'Time': Timestamp('2022-12-06 13:04:58.140737'), 'Ticker': 'HEROMOTOCO', 'Close': 2792.8}
{'Time': Timestamp('2022-12-06 13:04:58.142014'), 'Ticker': 'ZEEL', 'Close': 265.05}
{'Time': Timestamp('2022-12-06 13:04:58.142830'), 'Ticker': 'ADANIENT', 'Close': 4016.0}
{'Time': Timestamp('2022-12-06 13:04:58.143588'), 'Ticker': 'HCLTECH', 'Close': 1114.35}
{'Time': Timestamp('2022-12-06 13:04:58.144369'), 'Ticker': 'HDFCLIFE', 'Close': 587.5}
{'Time': Timestamp('2022-12-06 13:04:58.145126'), 'Ticker': 'INDUSINDBK', 'Close': 1186.2}
{'Time': Timestamp('2022-12-06 13:04:58.146154'), 'Ticker': 'SBIN', 'Close': 614.45}
{'Time': Timestamp('2022-12-06 13:04:58.147398'), 'Ticker': '

...

{'Time': Timestamp('2022-12-06 13:49:58.928741'), 'Ticker': 'HDFCBANK', 'Close': 1609.95}
{'Time': Timestamp('2022-12-06 13:49:58.931868'), 'Ticker': 'KOTAKBANK', 'Close': 1933.1}
{'Time': Timestamp('2022-12-06 13:49:58.934403'), 'Ticker': 'PNB', 'Close': 55.8}
{'Time': Timestamp('2022-12-06 13:49:58.936984'), 'Ticker': 'RBLBANK', 'Close': 153.65}
{'Time': Timestamp('2022-12-06 13:49:58.938358'), 'Ticker': 'ITC', 'Close': 337.4}
{'Time': Timestamp('2022-12-06 13:49:58.939215'), 'Ticker': 'SHREECEM', 'Close': 24000.0}
{'Time': Timestamp('2022-12-06 13:49:58.940052'), 'Ticker': 'BEL', 'Close': 105.7}
{'Time': Timestamp('2022-12-06 13:49:58.942049'), 'Ticker': 'ADANIPORTS', 'Close': 898.5}
{'Time': Timestamp('2022-12-06 13:49:58.942410'), 'Ticker': 'DRREDDY', 'Close': 4399.0}
{'Time': Timestamp('2022-12-06 13:49:58.943444'), 'Ticker': 'AXISBANK', 'Close': 900.5}
{'Time': Timestamp('2022-12-06 13:49:58.945043'), 'Ticker': 'CIPLA', 'Close': 1105.85}
{'Time': Timestamp('2022-12-06 13:49:58.9

{'Time': Timestamp('2022-12-06 13:50:59.580438'), 'Ticker': 'LICHSGFIN', 'Close': 404.4}
{'Time': Timestamp('2022-12-06 13:50:59.589318'), 'Ticker': 'INDHOTEL', 'Close': 327.4}
-------------------------START OF 51 mins-----------------------
{'Time': Timestamp('2022-12-06 13:51:58.393175'), 'Ticker': 'PNB', 'Close': 55.7}
{'Time': Timestamp('2022-12-06 13:51:58.395529'), 'Ticker': 'RELIANCE', 'Close': 2693.0}
{'Time': Timestamp('2022-12-06 13:51:58.396357'), 'Ticker': 'TITAN', 'Close': 2599.3}
{'Time': Timestamp('2022-12-06 13:51:58.396961'), 'Ticker': 'ASIANPAINT', 'Close': 3148.3}
{'Time': Timestamp('2022-12-06 13:51:58.397457'), 'Ticker': 'BAJAJFINSV', 'Close': 1632.6}
{'Time': Timestamp('2022-12-06 13:51:58.397938'), 'Ticker': 'GRASIM', 'Close': 1813.0}
{'Time': Timestamp('2022-12-06 13:51:58.398398'), 'Ticker': 'RBLBANK', 'Close': 153.5}
{'Time': Timestamp('2022-12-06 13:51:58.398891'), 'Ticker': 'ASHOKLEY', 'Close': 145.6}
{'Time': Timestamp('2022-12-06 13:51:58.399380'), 'Ticker

{'Time': Timestamp('2022-12-06 13:52:58.504069'), 'Ticker': 'HDFC', 'Close': 2671.1}
{'Time': Timestamp('2022-12-06 13:52:58.505942'), 'Ticker': 'IRCTC', 'Close': 718.7}
{'Time': Timestamp('2022-12-06 13:52:58.507203'), 'Ticker': 'WIPRO', 'Close': 411.1}
{'Time': Timestamp('2022-12-06 13:52:58.508867'), 'Ticker': 'CUMMINSIND', 'Close': 1447.9}
{'Time': Timestamp('2022-12-06 13:52:58.509319'), 'Ticker': 'KOTAKBANK', 'Close': 1930.8}
{'Time': Timestamp('2022-12-06 13:52:58.509788'), 'Ticker': 'PNB', 'Close': 55.7}
{'Time': Timestamp('2022-12-06 13:52:58.510793'), 'Ticker': 'RELIANCE', 'Close': 2693.95}
{'Time': Timestamp('2022-12-06 13:52:58.511525'), 'Ticker': 'RBLBANK', 'Close': 153.65}
{'Time': Timestamp('2022-12-06 13:52:58.511922'), 'Ticker': 'ONGC', 'Close': 141.95}
{'Time': Timestamp('2022-12-06 13:52:58.512983'), 'Ticker': 'SHREECEM', 'Close': 23997.2}
{'Time': Timestamp('2022-12-06 13:52:58.513931'), 'Ticker': 'TATAMOTORS', 'Close': 427.75}
{'Time': Timestamp('2022-12-06 13:52:5

{'Time': Timestamp('2022-12-06 13:53:59.579846'), 'Ticker': 'DLF', 'Close': 413.7}
{'Time': Timestamp('2022-12-06 13:53:59.582576'), 'Ticker': 'ULTRACEMCO', 'Close': 7214.65}
-------------------------START OF 54 mins-----------------------
{'Time': Timestamp('2022-12-06 13:54:58.004747'), 'Ticker': 'ASIANPAINT', 'Close': 3148.45}
{'Time': Timestamp('2022-12-06 13:54:58.006363'), 'Ticker': 'BAJAJFINSV', 'Close': 1633.2}
{'Time': Timestamp('2022-12-06 13:54:58.007436'), 'Ticker': 'GRASIM', 'Close': 1814.75}
{'Time': Timestamp('2022-12-06 13:54:58.007848'), 'Ticker': 'RELIANCE', 'Close': 2693.7}
{'Time': Timestamp('2022-12-06 13:54:58.008195'), 'Ticker': 'BAJAJ-AUTO', 'Close': 3677.65}
{'Time': Timestamp('2022-12-06 13:54:58.008549'), 'Ticker': 'RBLBANK', 'Close': 153.75}
{'Time': Timestamp('2022-12-06 13:54:58.008877'), 'Ticker': 'ASHOKLEY', 'Close': 145.8}
{'Time': Timestamp('2022-12-06 13:54:58.009228'), 'Ticker': 'TVSMOTOR', 'Close': 1030.0}
{'Time': Timestamp('2022-12-06 13:54:58.009

{'Time': Timestamp('2022-12-06 13:55:59.064181'), 'Ticker': 'SBICARD', 'Close': 819.9}
{'Time': Timestamp('2022-12-06 13:55:59.067929'), 'Ticker': 'BHARATFORG', 'Close': 840.0}
{'Time': Timestamp('2022-12-06 13:55:59.068769'), 'Ticker': 'CIPLA', 'Close': 1106.05}
{'Time': Timestamp('2022-12-06 13:55:59.070124'), 'Ticker': 'FEDERALBNK', 'Close': 135.35}
{'Time': Timestamp('2022-12-06 13:55:59.071854'), 'Ticker': 'BAJFINANCE', 'Close': 6688.55}
{'Time': Timestamp('2022-12-06 13:55:59.072543'), 'Ticker': 'HDFCLIFE', 'Close': 587.8}
{'Time': Timestamp('2022-12-06 13:55:59.072900'), 'Ticker': 'INDUSINDBK', 'Close': 1188.7}
{'Time': Timestamp('2022-12-06 13:55:59.073587'), 'Ticker': 'TATACHEM', 'Close': 1043.65}
{'Time': Timestamp('2022-12-06 13:55:59.074551'), 'Ticker': 'IRCTC', 'Close': 718.2}
{'Time': Timestamp('2022-12-06 13:55:59.074903'), 'Ticker': 'MARUTI', 'Close': 8714.65}
{'Time': Timestamp('2022-12-06 13:55:59.077436'), 'Ticker': 'BHARTIARTL', 'Close': 834.05}
{'Time': Timestamp('